In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import folium
from folium.plugins import PolyLineTextPath
import numpy as np
import matplotlib.colors as mcolors

In [2]:

# Load data
routine_df = pd.read_csv('routine.csv')
polygon_df = pd.read_csv('polygon.csv')

In [3]:
# Convert WKT to geometries and create GeoDataFrames 
polygon_df['geometry'] = polygon_df['wkt'].apply(wkt.loads)

#! EPSG:4326  -->  CRS code from the EPSG (European Petroleum Survey Group) database.
polygon_gdf = gpd.GeoDataFrame(polygon_df, geometry='geometry', crs='EPSG:4326')
routine_gdf = gpd.GeoDataFrame(routine_df, geometry=gpd.points_from_xy(routine_df['lon'], routine_df['lat']), crs='EPSG:4326')

# Spatial join  (sjoin) to find points within polygons
# Join points with polygons
joined_gdf = gpd.sjoin(routine_gdf, polygon_gdf, how='left', predicate='within')

# Merge municipality names 
routine_df = routine_df.join(joined_gdf[['nome_com']], how='left')

In [4]:


###############################################################################################
###############################################################################################
#  Create base map
m = folium.Map(location=[routine_df['lat'].mean(), routine_df['lon'].mean()], zoom_start=12)
###############################################################################################
###############################################################################################

In [5]:
# Generate random color map for municipalities
def generate_random_color():
    return mcolors.to_hex(np.random.rand(3,))

color_map = {name: generate_random_color() for name in routine_df['nome_com'].dropna().unique()}

# Add points to the map with colors based on municipalities
for _, row in routine_df.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        color=color_map.get(row['nome_com'], 'blue'),
        fill=True,
        fill_color=color_map.get(row['nome_com'], 'blue')
    ).add_to(m)

# Add polyline and direction arrows
coordinates = list(zip(routine_df['lat'], routine_df['lon']))
line = folium.PolyLine(locations=coordinates, color='blue').add_to(m)
PolyLineTextPath(line, ' ➜ ', repeat=True, offset=7, attributes={'fill': 'blue'}).add_to(m)

# Save the map as an HTML file
m.save('map_with_colored_points.html')
